In [ ]:
import plotly.offline as ply
#ply.init_notebook_mode()

import numpy as np
import scipy as sp
from scipy.special import binom
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly
import pandas as pd
import mpnum as mp
import sklearn

from sklearn import svm
from numba import autojit

import plotly.plotly as py
from plotly.graph_objs import *

#%matplotlib notebook

# Sweeeeeping!!

Define Ausiliar functions

In [ ]:
@autojit
def feature_map(x):
    """Local feature map"""
    
    x_arr = 0.5 * np.pi * np.array([x]*d)
    s = np.arange(d)
    return np.sqrt(binom([d-1]*d,s)) * np.power(np.cos(x_arr), d-1-s) * np.power(np.sin(x_arr), s)

@autojit
def Tdelta(l):
    """Create a tensor kronecker delta"""
    return mp.MPArray.from_kron([np.array([1-l,l])])

@autojit
def evaluate(Tweight, x, y):
    xy_mpa = mp.MPArray.from_kron([feature_map(x),feature_map(y)]).group_sites(2) # feature tensor
    Tf = mp.dot(Tweight,xy_mpa,axes=([1,2],[0,1]))
    W0 = mp.dot(Tf, Tdelta(0)).to_array()
    W1 = mp.dot(Tf, Tdelta(1)).to_array()
    Wdiff = W0 - W1
    Wsum = W0 + W1
    return Wdiff * Wsum / np.abs(Wdiff * Wsum)

def plot_tn_decision_function(Tweight, ax=None):
    """Plot the decision function for a 2D SVC"""
    if ax is None:
        ax = plt.gca()
    x = np.linspace(plt.xlim()[0], plt.xlim()[1], 100)
    y = np.linspace(plt.ylim()[0], plt.ylim()[1], 100)
    Y, X = np.meshgrid(y, x)
    P = np.zeros_like(X)
    for i, xi in enumerate(x):
        for j, yj in enumerate(y):
            P[i, j] =evaluate(Tweight,xi,yj)
    # plot the margins
    ax.contour(X, Y, P, colors='k',
               levels=[-1, 0, 1], alpha=0.5,
               linestyles=['--', '-', '--'])

def plot_svc_decision_function(clf, ax=None):
    """Plot the decision function for a 2D SVC"""
    if ax is None:
        ax = plt.gca()
    x = np.linspace(plt.xlim()[0], plt.xlim()[1], 100)
    y = np.linspace(plt.ylim()[0], plt.ylim()[1], 100)
    Y, X = np.meshgrid(y, x)
    P = np.zeros_like(X)
    for i, xi in enumerate(x):
        for j, yj in enumerate(y):
            P[i, j] = clf.decision_function(np.c_[xi.ravel(), yj.ravel()])
    # plot the margins
    ax.contour(X, Y, P, colors='k',
               levels=[-1, 0, 1], alpha=0.5,
               linestyles=['--', '-', '--'])

Define sweeping algorithm

In [ ]:
@autojit
def update2sites(data, label, Tweight):    
    TdeltaB = mp.MPArray.from_kron([np.array([0]*2), np.array([0]*d), np.array([0]*d)]).group_sites(3) #null tensor deltaB (single size)
    for idx in range(len(data)):
        # create the full feature map
        xy_mpa = mp.MPArray.from_kron([feature_map(data[idx][0]),feature_map(data[idx][1])]).group_sites(2) # feature tensor
        # tensot product between the weights and the feature map
        Tf = mp.dot(Tweight,xy_mpa,axes=([1,2],[0,1])) # the last two physical legs are of Tweight are contracted with the feature tensor
        Tcoef = (Tdelta(label[idx]) - Tf)
        Ttemp = mp.MPArray.from_kron([Tcoef.to_array(), xy_mpa.to_array()]).group_sites(2)
        TdeltaB = TdeltaB + Ttemp
    
    Tweight = Tweight + alpha * TdeltaB
    return Tweight

# Classify data

## Two overlapping  gaussians

In [ ]:
mean1=[0.3,0.3]
cov1=[[0.001,0],[0,0.001]]
mean2=[0.4,0.4]
cov2=[[0.001,0],[0,0.001]]
class0=np.random.multivariate_normal(mean1, cov1, 10000)
class1=np.random.multivariate_normal(mean2, cov2, 10000)

f1=plt.figure()
plt.plot(class0.T[0],class0.T[1],".")
plt.plot(class1.T[0],class1.T[1],"x")
plt.show()

Prepare training data

In [ ]:
# set data
numdata = 1000
train_data = np.concatenate([class0[0:numdata], class1[0:numdata]])
label = np.concatenate([np.array([0]*numdata), np.array([1]*numdata)])

### Tensor network

Generate and update a random tensor weight using gradient descent steps

In [ ]:
d = 2 # local dimension
rng = np.random.RandomState(seed=143)
Tweight = mp.random_mpa(sites=1, ldim=[[2,d,d]], bdim=1, randstate=rng, normalized=True)

In [ ]:
numsteps = 100
alpha = 5e-4 # control convergence

for idx in range(numsteps):
    Tweight = update2sites(train_data, label, Tweight)

Plot results

In [ ]:
point_to_plot = 10000
plt.plot(class0[0:point_to_plot,0], class0[0:point_to_plot,1], '.')
plt.plot(class1[0:point_to_plot,0], class1[0:point_to_plot,1], '.')
plot_tn_decision_function(Tweight, ax=None)
plt.show()

### SVM

In [ ]:
#Assumed you have, X (predictor) and Y (target) for training data set and x_test(predictor) of test_dataset
# Create SVM classification object 
model = svm.SVC(kernel='linear', C=4, gamma=1) 
# there is various option associated with it, like changing kernel, gamma and C value. Will discuss more # about it in next section.Train the model using the training sets and check score
model.fit(train_data, label)
model.score(train_data, label)

In [ ]:
point_to_plot = 10000
plt.plot(class0[0:point_to_plot,0], class0[0:point_to_plot,1], '.')
plt.plot(class1[0:point_to_plot,0], class1[0:point_to_plot,1], '.')
plot_svc_decision_function(model)
plt.show()

## Semi circular uniform distributions

In [ ]:
max_data = 10000
class0_radius = np.random.uniform(0, 0.5, max_data)
class0_phase = np.random.uniform(0, 0.5*np.pi, max_data)
class1_radius = np.random.uniform(0.5, 1, max_data)
class1_phase = np.random.uniform(0, 0.5*np.pi, max_data)

x0 = np.array([class0_radius*np.cos(class0_phase)])
y0 = np.array([class0_radius*np.sin(class0_phase)])
class0 = np.concatenate((x0.T, y0.T), axis=1)

x1 = np.array([class1_radius*np.cos(class1_phase)])
y1 = np.array([class1_radius*np.sin(class1_phase)])
class1 = np.concatenate((x1.T, y1.T), axis=1)

plt.plot(class0[:,0], class0[:,1], '.')
plt.plot(class1[:,0], class1[:,1], '.')
plt.show()

Prepare training data

In [ ]:
# set data
numdata = 1000
train_data = np.concatenate([class0[0:numdata],class1[0:numdata]])
label = np.concatenate([np.array([0]*numdata),np.array([1]*numdata)])

### Tensor network

Generate and update a random tensor weight using gradient descent steps

In [ ]:
d = 3
rng = np.random.RandomState(seed=143)
Tweight = mp.random_mpa(sites=1, ldim=[[2,d,d]], bdim=1, randstate=rng, normalized=True)

In [ ]:
numsteps = 200
alpha = 1e-3 # control convergence

for idx in range(numsteps):
    Tweight = update2sites(train_data, label, Tweight)

Plot results

In [ ]:
point_to_plot = 1000
plt.plot(class0[0:point_to_plot,0], class0[0:point_to_plot,1], '.')
plt.plot(class1[0:point_to_plot,0], class1[0:point_to_plot,1], '.')
plot_tn_decision_function(Tweight, ax=None)
plt.show()

### SVM

In [ ]:
#Assumed you have, X (predictor) and Y (target) for training data set and x_test(predictor) of test_dataset
# Create SVM classification object 
model = svm.SVC(kernel='poly', C=4, gamma=1) 
# there is various option associated with it, like changing kernel, gamma and C value. Will discuss more # about it in next section.Train the model using the training sets and check score
model.fit(train_data, label)
model.score(train_data, label)

In [ ]:
point_to_plot = 1000
plt.plot(class0[0:point_to_plot,0], class0[0:point_to_plot,1], '.')
plt.plot(class1[0:point_to_plot,0], class1[0:point_to_plot,1], '.')
plot_svc_decision_function(model)
plt.show()

## Circular distribution

In [ ]:
max_data = 10000
class0_radius = np.random.uniform(0, 0.5, max_data)
class0_phase = np.random.uniform(0, 2*np.pi, max_data)
class1_radius = np.random.uniform(0.5, 1, max_data)
class1_phase = np.random.uniform(0, 2*np.pi, max_data)

x0 = np.array([class0_radius*np.cos(class0_phase)]) * 0.5 + 0.5
y0 = np.array([class0_radius*np.sin(class0_phase)]) * 0.5 + 0.5
class0 = np.concatenate((x0.T, y0.T), axis=1)

x1 = np.array([class1_radius*np.cos(class1_phase)]) * 0.5 + 0.5
y1 = np.array([class1_radius*np.sin(class1_phase)]) * 0.5 + 0.5
class1 = np.concatenate((x1.T, y1.T), axis=1)

plt.plot(class0[:,0], class0[:,1], '.')
plt.plot(class1[:,0], class1[:,1], '.')
plt.show()

Prepare training data

In [ ]:
# set data
numdata = 1000
train_data = np.concatenate([class0[0:numdata],class1[0:numdata]])
label = np.concatenate([np.array([0]*numdata),np.array([1]*numdata)])

### Tensor network

Generate and update a random tensor weight using gradient descent steps

In [ ]:
d = 3
rng = np.random.RandomState(seed=143)
Tweight = mp.random_mpa(sites=1, ldim=[[2,d,d]], bdim=1, randstate=rng, normalized=True)

In [ ]:
numsteps = 200
alpha = 1e-3 # control convergence

for idx in range(numsteps):
    Tweight = update2sites(train_data, label, Tweight)

Plot results

In [ ]:
point_to_plot = 1000
plt.plot(class0[0:point_to_plot,0], class0[0:point_to_plot,1], '.')
plt.plot(class1[0:point_to_plot,0], class1[0:point_to_plot,1], '.')
plot_tn_decision_function(Tweight, ax=None)
plt.show()

### SVM

In [ ]:
#Assumed you have, X (predictor) and Y (target) for training data set and x_test(predictor) of test_dataset
# Create SVM classification object 
model = svm.SVC(kernel='rbf', C=4, gamma=1) 
# there is various option associated with it, like changing kernel, gamma and C value. Will discuss more # about it in next section.Train the model using the training sets and check score
model.fit(train_data, label)
model.score(train_data, label)

In [ ]:
point_to_plot = 1000
plt.plot(class0[0:point_to_plot,0], class0[0:point_to_plot,1], '.')
plt.plot(class1[0:point_to_plot,0], class1[0:point_to_plot,1], '.')
plot_svc_decision_function(model)
plt.show()

## Spiral distributions

In [ ]:
max_data = 10000
max_angle = 3*np.pi
theta = np.random.uniform(0, max_angle, max_data)

class0_phase = np.random.uniform(0, np.pi, max_data)
class1_phase = np.random.uniform(0, -np.pi, max_data)

x0 = np.array([theta * np.cos(theta + class0_phase)]) * 0.5 / max_angle + 0.5
y0 = np.array([theta * np.sin(theta + class0_phase)]) * 0.5 / max_angle + 0.5
class0 = np.concatenate((x0.T, y0.T), axis=1)

x1 = np.array([theta * np.cos(theta + class1_phase)]) * 0.5 / max_angle + 0.5
y1 = np.array([theta * np.sin(theta + class1_phase)]) * 0.5 / max_angle + 0.5
class1 = np.concatenate((x1.T, y1.T), axis=1)

plt.plot(class0[:,0], class0[:,1], '.')
plt.plot(class1[:,0], class1[:,1], '.')
plt.show()

Prepare training data

In [ ]:
# set data
numdata = 1000
train_data = np.concatenate([class0[0:numdata],class1[0:numdata]])
label = np.concatenate([np.array([0]*numdata),np.array([1]*numdata)])

### Tensor network

Generate and update a random tensor weight using gradient descent steps

In [ ]:
d = 70
rng = np.random.RandomState(seed=143)
Tweight = mp.random_mpa(sites=1, ldim=[[2,d,d]], bdim=1, randstate=rng, normalized=True)

In [ ]:
numsteps = 100
alpha = 1e-3 # control convergence

for idx in range(numsteps):
    Tweight = update2sites(train_data, label, Tweight)

Plot results

In [ ]:
point_to_plot = 1000
plt.plot(class0[0:point_to_plot,0], class0[0:point_to_plot,1], '.')
plt.plot(class1[0:point_to_plot,0], class1[0:point_to_plot,1], '.')
plot_tn_decision_function(Tweight, ax=None)
plt.show()

### SVM

In [ ]:
#Assumed you have, X (predictor) and Y (target) for training data set and x_test(predictor) of test_dataset
# Create SVM classification object 
model = svm.SVC(kernel='rbf', C=4, gamma=1) 
# there is various option associated with it, like changing kernel, gamma and C value. Will discuss more # about it in next section.Train the model using the training sets and check score
model.fit(train_data, label)
model.score(train_data, label)

In [ ]:
point_to_plot = 1000
plt.plot(class0[0:point_to_plot,0], class0[0:point_to_plot,1], '.')
plt.plot(class1[0:point_to_plot,0], class1[0:point_to_plot,1], '.')
plot_svc_decision_function(model)
plt.show()

# Real Sweeeeeping!

In [ ]:
@autojit
def feature_map(x):
    """Local feature map"""
    
    x_arr = 0.5 * np.pi * np.array([x]*d)
    s = np.arange(d)
    return np.sqrt(binom([d-1]*d,s)) * np.power(np.cos(x_arr), d-1-s) * np.power(np.sin(x_arr), s)

@autojit
def Tdelta(l):
    """Create a tensor kronecker delta"""
    return mp.MPArray.from_kron([np.array([1-l,l])])

@autojit
def evaluate(Tweight, x, y, z):
    xyz_mpa = mp.MPArray.from_kron([feature_map(x),feature_map(y),feature_map(z)]).group_sites(3) # feature tensor
    Tweight = Tweight.group_sites(3)
    Tf = mp.dot(Tweight,xyz_mpa,axes=([1,2,3],[0,1,2]))
    W0 = mp.dot(Tf, Tdelta(0)).to_array()
    W1 = mp.dot(Tf, Tdelta(1)).to_array()
    Wdiff = W0 - W1
    Wsum = W0 + W1
    return Wdiff * Wsum / np.abs(Wdiff * Wsum)

def tn_decision_function(Tweight, class0, class1):
    val0 = []
    for v0 in class0:
        val0.append(evaluate(Tweight, v0[0], v0[1], v0[2]))
    
    val1 = []
    for v1 in class1:
        val1.append(evaluate(Tweight, v1[0], v1[1], v1[2]))
    
    return val0, val1
    

In [ ]:
d = 2
m = 1
l = 1
sites = 3

rng = np.random.RandomState(seed=143)
ldim = [[d]]*(sites-1)
ldim.append([l,d])
Tweight = mp.random_mpa(sites=sites, ldim=ldim[::-1], bdim=m, randstate=rng, normalized=True)
print(Tweight.pdims)
print(Tweight.bdims)
print(Tweight.to_array())
a,b=Tweight.split(0)
print('---')
print(a.pdims)
print(a.to_array())
print(b.pdims)
print(b.to_array())
r=mp.dot(a,b.group_sites(2),axes=(2,0))

print(r.pdims)
print(r.to_array())


In [ ]:
#two gaussians 3D
mean1=[0.3,0.3,0.3]
cov1=[[0.001,0,0],[0,0.001,0],[0,0,0.001]]
mean2=[0.4,0.4,0.4]
cov2=[[0.001,0,0],[0,0.001,0],[0,0,0.001]]
class0=np.random.multivariate_normal(mean1, cov1, 1)
class1=np.random.multivariate_normal(mean2, cov2, 1)

trace1 = Scatter3d(
    x=class0.T[0],
    y=class0.T[1],
    z=class0.T[2],
    mode='markers',
    marker=dict(
        size=3,
        symbol = "cross",
        line=dict(
            width=0.5
        )
    )
)
trace2 = Scatter3d(
    x=class1.T[0],
    y=class1.T[1],
    z=class1.T[2],
    mode='markers',
    marker=dict(
        #color='rgb(127, 127, 127)',
        size=3,
        symbol='circle',
        line=dict(
            width=1
        )
    )
)

data_plot = [trace1, trace2]
layout = Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = Figure(data=data_plot, layout=layout)
ply.plot(fig, filename='simple-3d-scatter')

In [ ]:
def split_sites(mpa, site_label):
    """Split physical legs of the site site_label so to have one leg per site"""
    
    max_num_site = len(t1)
    if site_label > max_num_site - 1:
        print("Site does not exist")
        return
    if site_label < 0:
        print("site must be non negative")
        return
    
    if site_label > 0:
        mpa_left, mpa_center = mpa.split(site_label - 1)
    else:
        mpa_center = mpa
        mpa_left = []
        
    if site_label < max_num_site - 1:
        mpa_center, mpa_right = mpa_center.split(0)
    else:
        mpa_right = []
    
    mpa_center_nlegs = mpa_center.plegs[0]
    mpa_center = mpa_center.split_sites(mpa_center.plegs[0])
    mpa_list = [mpa_left, mpa_center, mpa_right]
    mpa_list = [x for x in mpa_list if x]        # get rid of empty list
    mpa_splitted = mp.outer(mpa_list)
    
    if site_label < max_num_site - 1:
        mpa_splitted = mpa_splitted.pleg2bleg(site_label - 1 + mpa_center_nlegs)
    if site_label > 0:
        mpa_splitted = mpa_splitted.pleg2bleg(site_label - 1)

    return mp.prune(mpa_splitted)

In [ ]:
rng = np.random.RandomState(seed=143)
t1=mp.random_mpa(sites=2, ldim=[[3,4],[4,2]], bdim=2, randstate=rng, normalized=True)
site_to_split=1
print(t1.pdims)
mpa = split_sites(t1,site_to_split)
print(mpa.pdims)
print(mpa.reverse().pdims)

In [ ]:
#two gaussians 4D
mean1=[0.3,0.3,0.3,0.3]
cov1=[[0.001,0,0,0],[0,0.001,0,0],[0,0,0.001,0],[0,0,0,0.001]]
mean2=[0.4,0.4,0.4,0.4]
cov2=[[0.001,0,0,0],[0,0.001,0,0],[0,0,0.001,0],[0,0,0,0.001]]
class0=np.random.multivariate_normal(mean1, cov1, 100)
class1=np.random.multivariate_normal(mean2, cov2, 100)

@autojit
def Tdelta(l):
    """Create a tensor kronecker delta"""
    return mp.MPArray.from_kron([np.array([1-l,l,0])])

d = 4
m = 2
l = 3
sites = 3
alpha = 0.1

rng = np.random.RandomState(seed=143)
ldim = [[d]]*(sites-1)
ldim.append([l,d])
TW = mp.random_mpa(sites=sites, ldim=ldim[::-1], bdim=m, randstate=rng, normalized=True)

numdata = 10
train_data = np.concatenate([class0[0:numdata],class1[0:numdata]])
label = np.concatenate([np.array([0]*numdata),np.array([1]*numdata)])

#bond 0  
bond = 0
B, TW_right = TW.split(1)
B = B.group_sites(2)
TdeltaB = mp.MPArray.from_kron([np.array([0]*l), np.array([0]*d), np.array([0]*d), np.array([0]*m)]).group_sites(4) #null tensor deltaB (single size
for idx in range(2):
    TPhi = mp.MPArray.from_kron([feature_map(train_data[idx][0]), feature_map(train_data[idx][1])]) # feature tensor
    TPhi_right = mp.MPArray.from_kron([feature_map(train_data[idx][n_site]) for n_site in range(2, sites)])

    TWPhi_right = mp.prune(mp.dot(TW_right, TPhi_right, axes=(-1,0)))
    TPhiTilde = mp.outer([TPhi, TWPhi_right]).group_sites(3)
    
    Tf = mp.prune(mp.dot(B, TPhiTilde, axes=([1,2,3],[0,1,2])))
    
    Tcoef = (Tdelta(label[idx]) - Tf)
    Ttemp = mp.outer([Tcoef, TPhiTilde]).group_sites(2)
    TdeltaB = TdeltaB + Ttemp

B = B + alpha * TdeltaB

B = B.transpose([1,0,2,3])
B = B.split_sites(4)
Bl, Br = B.split(0)
Br = Br.group_sites(3)
Bsvd = mp.outer([Bl,Br]).pleg2bleg(0)
Bsvd.compress(method='svd', bdim=m)

#bond 1
bond = 1
TW_left, Bl = Bsvd.split(0)
B = mp.prune(mp.outer([Bl,TW_right]).pleg2bleg(0)).group_sites(2)

TdeltaB = mp.MPArray.from_kron([np.array([0]*m), np.array([0]*l), np.array([0]*d), np.array([0]*d)]).group_sites(4) #null tensor deltaB (single size
for idx in range(2):
    TPhi = mp.MPArray.from_kron([feature_map(train_data[idx][1]), feature_map(train_data[idx][2])]) # feature tensor
    TPhi_left = mp.MPArray.from_kron([feature_map(train_data[idx][0])])

    TWPhi_left = mp.prune(mp.dot(TW_left, TPhi_left, axes=(0,0)))
    TPhiTilde = mp.outer([TWPhi_left, TPhi]).group_sites(3)
    
    Tf = mp.prune(mp.dot(B, TPhiTilde, axes=([0,2,3],[0,1,2])))
    
    Tcoef = (Tdelta(label[idx]) - Tf)
    Ttemp = mp.outer([Tcoef, TPhiTilde]).group_sites(2).transpose([1,0,2,3])
    TdeltaB = TdeltaB + Ttemp

B = B + alpha * TdeltaB
B = B.transpose([0,2,1,3])
B = B.split_sites(4)
Bl, Br = B.split(1)
Bl = Bl.group_sites(2)
Br = Br.group_sites(2)
Bsvd = mp.outer([Bl,Br]).pleg2bleg(0)
Bsvd.compress(method='svd', bdim=m)

TW = mp.outer([TW_left,Bsvd]).pleg2bleg(0)
TW = TW.reverse()

# update Weight with reversed algorithm
#bond 0  
bond = 0
B, TW_right = TW.split(1)
B = B.group_sites(2)
TdeltaB = mp.MPArray.from_kron([np.array([0]*l), np.array([0]*d), np.array([0]*d), np.array([0]*m)]).group_sites(4) #null tensor deltaB (single size
for idx in range(2):
    TPhi = mp.MPArray.from_kron([feature_map(train_data[idx][2]), feature_map(train_data[idx][1])]) # feature tensor
    TPhi_right = mp.MPArray.from_kron([feature_map(train_data[idx][n_site]) for n_site in range(2, sites)])

    TWPhi_right = mp.prune(mp.dot(TW_right, TPhi_right, axes=(-1,0)))
    TPhiTilde = mp.outer([TPhi, TWPhi_right]).group_sites(3)
    
    Tf = mp.prune(mp.dot(B, TPhiTilde, axes=([1,2,3],[0,1,2])))
    
    Tcoef = (Tdelta(label[idx]) - Tf)
    Ttemp = mp.outer([Tcoef, TPhiTilde]).group_sites(2)
    TdeltaB = TdeltaB + Ttemp

B = B + alpha * TdeltaB
B = B.transpose([1,0,2,3])
B = B.split_sites(4)
Bl, Br = B.split(0)
Br = Br.group_sites(3)
Bsvd = mp.outer([Bl,Br]).pleg2bleg(0)
Bsvd.compress(method='svd', bdim=m)

#bond 1
bond = 1
TW_left, Bl = Bsvd.split(0)
B = mp.prune(mp.outer([Bl,TW_right]).pleg2bleg(0)).group_sites(2)

TdeltaB = mp.MPArray.from_kron([np.array([0]*m), np.array([0]*l), np.array([0]*d), np.array([0]*d)]).group_sites(4) #null tensor deltaB (single size
for idx in range(2):
    TPhi = mp.MPArray.from_kron([feature_map(train_data[idx][1]), feature_map(train_data[idx][0])]) # feature tensor
    TPhi_left = mp.MPArray.from_kron([feature_map(train_data[idx][0])])

    TWPhi_left = mp.prune(mp.dot(TW_left, TPhi_left, axes=(0,0)))
    TPhiTilde = mp.outer([TWPhi_left, TPhi]).group_sites(3)
    
    Tf = mp.prune(mp.dot(B, TPhiTilde, axes=([0,2,3],[0,1,2])))
    
    Tcoef = (Tdelta(label[idx]) - Tf)
    Ttemp = mp.outer([Tcoef, TPhiTilde]).group_sites(2).transpose([1,0,2,3])
    TdeltaB = TdeltaB + Ttemp

B = B + alpha * TdeltaB
B = B.transpose([0,2,1,3])
B = B.split_sites(4)
Bl, Br = B.split(1)
Bl = Bl.group_sites(2)
Br = Br.group_sites(2)
Bsvd = mp.outer([Bl,Br]).pleg2bleg(0)
Bsvd.compress(method='svd', bdim=m)

TW = mp.outer([TW_left,Bsvd]).pleg2bleg(0)
TW = TW.reverse()

#mpa2 = mp.MPArray.from_kron([np.array([1,0,0,0]),np.array([1,0,0,0]),np.array([1,0,0,0])])
#a=mp.dot(TW, mpa2)
#print(TdeltaB.pdims)
print(TW.pdims)

In [ ]:
#two gaussians 3D
d = 4
m = 2
l = 3
sites = 3
alpha = 0.1

rng = np.random.RandomState(seed=143)
ldim = [[d]]*(sites-1)
ldim.append([l,d])
#TW = mp.random_mpa(sites=sites, ldim=ldim[::-1], bdim=m, randstate=rng, normalized=True)

mean1=[0.3,0.3,0.3]
cov1=[[0.001,0,0],[0,0.001,0],[0,0,0.001]]
mean2=[0.4,0.4,0.4]
cov2=[[0.001,0,0],[0,0.001,0],[0,0,0.001]]
class0=np.random.multivariate_normal(mean1, cov1, 1000)
class1=np.random.multivariate_normal(mean2, cov2, 1000)

cl0, cl1 = tn_decision_function(TW, class0, class1)

trace1 = Scatter3d(
    x=class0.T[0],
    y=class0.T[1],
    z=class0.T[2],
    mode='markers',
    marker=dict(
        size=10,
        symbol = "circle",
        color = (np.array(cl0) + 1)*0.5,
        colorscale=[[0, 'rgb(0,0,255)'], [1, 'rgb(255,0,0)']],
        line=dict(
            width=0.5
        )
    )
)
trace2 = Scatter3d(
    x=class1.T[0],
    y=class1.T[1],
    z=class1.T[2],
    mode='markers',
    marker=dict(
        #color='rgb(127, 127, 127)',
        size=3,
        symbol='circle',
        line=dict(
            width=1
        )
    )
)

data_plot = [trace1]#, trace2]
layout = Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = Figure(data=data_plot, layout=layout)
ply.iplot(fig, filename='simple-3d-scatter')